In [ ]:
import pandas as pd
import wrf
import glob
import numpy as np
import os
import netCDF4 as nc
from pylab import *
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from geographiclib.geodesic import Geodesic
Geo = Geodesic.WGS84

warnings.filterwarnings('ignore')

In [ ]:
filedir = '../Data/originals/'
filename = []
filename += glob.glob(filedir+os.sep+'*_cut.nc')

In [ ]:
ncdf = nc.Dataset(filename[0],engine='netcdf4')

In [ ]:
lons = wrf.getvar(ncdf, 'XLONG')
lats = wrf.getvar(ncdf, 'XLAT')

In [ ]:
df = pd.read_csv('tracks_new.csv', sep=',', index_col = 0)
df.reset_index(drop=True, inplace=True)
df['time'] = pd.to_datetime(df['time'])

In [ ]:
atm_times = np.load('times.npy')
atm_times = np.reshape(atm_times, 3551*8)
atm_times_pd = []

for i in range(atm_times.shape[0]):
    atm_times_pd.append(pd.Timestamp(atm_times[i]))

pl_times = list(df['time'])
both = set(atm_times_pd).intersection(pl_times)

df = df[df['datetime'].isin(both)]

In [ ]:
atm_indexes = np.load('indices_atm_times.npy')

In [ ]:
var_names = [
             'hfx',
             'mcao500',
             'mcao700',
             'Th500',
             'SST500',
             'T700',
             'wind10',
             'Th500_sst',
             'Th700',
             'Th850',
             'wind_t',
             'Th700_sst',
             'Th900_sst',
             'avo850',
             'pvo400'
             
]         

In [ ]:
# fig,ax = make_cartopy()
for i in range(len(var_names)):
    print(var_names[i])
    var = np.asarray(np.load(f'{var_names[i]}.npy'))
    var = var.reshape(2334*8, 83,162)
    var = var[atm_indexes, :,:]
    cycl_data = []
    cycl_masked = []
    masks = []
    t = 0
    for ind,group in df.groupby(['time']):
        lat = group.lat0.values # широта и долгота каждого центра циклона
        lon = group.lon0.values
        diam = group.diam.values
        for case in range(len(lon)):
            dist = np.zeros_like(lats)
            for xx in range(lats.shape[0]):
                for yy in range(lats.shape[1]): # lons, lats - это переменные массива ворфа
                    dist[xx,yy] = Geo.Inverse(lon[case], lat[case], lons[xx,yy], lats[xx,yy])['s12']       
            mt = np.ma.masked_where((dist/1000)>(diam[case]/2), var[t,:,:].reshape((83,162)))
            cycl_data.append(mt[mt.mask == False])
            masks.append(mt.mask)
                
        t += 1
    np.save(arr = np.asarray(cycl_data), file = f'{var_names[i]}_cycl_data_false.npy', allow_pickle = True)
#     np.save(arr = np.asarray(masks), file = 'masks_pl_fin.npy', allow_pickle = True)